In [1]:
import pandas as pd

from pyomo.environ import *
from pyomo.gdp import *

In [2]:
TASKS = {
    ('Wall1','Blue')   : {'dur': 45, 'prec': None},
    ('Wall1','Yellow') : {'dur': 10, 'prec': ('Wall1','Blue')},
    ('Wall2','Blue')   : {'dur': 20, 'prec': ('Wall2','Green')},
    ('Wall2','Green')  : {'dur': 10, 'prec': None},
    ('Wall2','Yellow') : {'dur': 34, 'prec': ('Wall2','Blue')},
    ('Wall3','Blue')   : {'dur': 12, 'prec': ('Wall3','Yellow')},
    ('Wall3','Green')  : {'dur': 17, 'prec': ('Wall3','Blue')},
    ('Wall3','Yellow') : {'dur': 28, 'prec': None},
}

In [3]:
def jobshop_model(TASKS):

    model = ConcreteModel()

    model.TASKS = Set(initialize = TASKS.keys(), dimen=2)
    model.JOBS = Set(initialize = list(set([j for (j,m) in model.TASKS])))
    model.MACHINES = Set(initialize = list(set([m for (j,m) in model.TASKS])))

    model.TASKORDER = Set(initialize = model.TASKS * model.TASKS, dimen=4,
        filter = lambda model, j, m, k, n: (k,n) == TASKS[(j,m)]['prec'])

    model.DISJUNCTIONS = Set(initialize = model.JOBS * model.JOBS * model.MACHINES, dimen=3,
        filter = lambda model, j, k, m: j < k and (j,m) in model.TASKS and (k,m) in model.TASKS)

    model.dur = Param(model.TASKS, initialize=lambda model, j, m: TASKS[(j,m)]['dur'])

    ub = sum([model.dur[j, m] for (j,m) in model.TASKS])

    # δημιουργώ μεταβλητές απόφασης
    model.makespan = Var(bounds=(0, ub))
    model.start = Var(model.TASKS, bounds=(0, ub))

    model.objective = Objective(expr = model.makespan, sense = minimize)

    model.finish = Constraint(model.TASKS, rule=lambda model, j, m:
        model.start[j,m] + model.dur[j,m] <= model.makespan)

    model.preceding = Constraint(model.TASKORDER, rule=lambda model, j, m, k, n:
        model.start[k,n] + model.dur[k,n] <= model.start[j,m])

    model.disjunctions = Disjunction(model.DISJUNCTIONS, rule=lambda model,j,k,m:
        [model.start[j,m] + model.dur[j,m] <= model.start[k,m],
         model.start[k,m] + model.dur[k,m] <= model.start[j,m]])

    TransformationFactory('gdp.hull').apply_to(model)
    return model

jobshop_model(TASKS)

In [4]:
def jobshop_solve(model):
    SolverFactory('gurobi').solve(model)

    results = [{'Job': j,
                'Machine': m,
                'Start': model.start[j, m](),
                'Duration': model.dur[j,m],
                'Finish': model.start[(j, m)]() + model.dur[j,m]}
               for j,m in model.TASKS]
    return results

def jobshop(TASKS):
    return jobshop_solve(jobshop_model(TASKS))

results = jobshop(TASKS)
results
schedule = pd.DataFrame(results)

print('Total time needed: ', ceil(schedule.Finish.max()))
print('\nSchedule by Job')
print(schedule.sort_values(by=['Job','Start']).set_index(['Job', 'Machine']))

print('\nSchedule by Machine')
print(schedule.sort_values(by=['Machine','Start']).set_index(['Machine', 'Job']))

Total time needed:  97

Schedule by Job
               Start  Duration  Finish
Job   Machine                         
Wall1 Blue      42.0        45    87.0
      Yellow    87.0        10    97.0
Wall2 Green      0.0        10    10.0
      Blue      10.0        20    30.0
      Yellow    30.0        34    64.0
Wall3 Yellow     0.0        28    28.0
      Blue      30.0        12    42.0
      Green     80.0        17    97.0

Schedule by Machine
               Start  Duration  Finish
Machine Job                           
Blue    Wall2   10.0        20    30.0
        Wall3   30.0        12    42.0
        Wall1   42.0        45    87.0
Green   Wall2    0.0        10    10.0
        Wall3   80.0        17    97.0
Yellow  Wall3    0.0        28    28.0
        Wall2   30.0        34    64.0
        Wall1   87.0        10    97.0
